1.문서 내용을 읽는다.
2. 문서를 쪼갠다
    - 토큰수 초과로 답변 생성하지 못할 가능성 존재
    - 인풋이 길면 답변 생성이 오래 걸린다
3. 임베딩 -> Vector DB에 저장
4. 질문을 할 시, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

-> 공식 문서 : https://docs.langchain.com/oss/python/langchain/overview , https://developers.openai.com/api/docs

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
#구분자를 리스트로 여러 개를 가질 수 있다.

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, #chunk size
    chunk_overlap=200, #chunk마다 곂치는 정도 -> 유사도가 높은 것들을 가져오기 위해
)

loader = Docx2txtLoader("./korail.docx")

data_list = loader.load_and_split(text_splitter=text_splitter)

data_list

[Document(metadata={'source': './korail.docx'}, page_content='한국철도공사법\n\n한국철도공사법\n\n[시행 2026. 1. 2.] [법률 제21065호, 2025. 10. 1., 타법개정]\n\n국토교통부(철도운영과) 044-201-4774\n\n\n\n제1조(목적) 이 법은 한국철도공사를 설립하여 철도 운영의 전문성과 효율성을 높임으로써 철도산업과 국민경제의 발전에 이바지함을 목적으로 한다.\n\n[전문개정 2009. 3. 25.]\n\n\n\n제2조(법인격) 한국철도공사(이하 “공사”라 한다)는 법인으로 한다.\n\n\n\n제3조(사무소) ① 공사의 주된 사무소의 소재지는 정관으로 정한다.\n\n② 공사는 업무수행을 위하여 필요하면 이사회의 의결을 거쳐 필요한 곳에 하부조직을 둘 수 있다.\n\n[전문개정 2009. 3. 25.]\n\n\n\n제4조(자본금 및 출자) ① 공사의 자본금은 22조원으로 하고, 그 전부를 정부가 출자한다.\n\n② 제1항에 따른 자본금의 납입 시기와 방법은 재정경제부장관이 정하는 바에 따른다.<개정 2025. 10. 1.>\n\n③ 국가는 「국유재산법」에도 불구하고 「철도산업발전 기본법」 제22조제1항제1호에 따른 운영자산을 공사에 현물로 출자한다.\n\n④ 제3항에 따라 국가가 공사에 출자를 할 때에는 「국유재산의 현물출자에 관한 법률」에 따른다.\n\n[전문개정 2009. 3. 25.]\n\n\n\n제5조(등기) ① 공사는 주된 사무소의 소재지에서 설립등기를 함으로써 성립한다.\n\n② 제1항에 따른 공사의 설립등기와 하부조직의 설치ㆍ이전 및 변경 등기, 그 밖에 공사의 등기에 필요한 사항은 대통령령으로 정한다.\n\n③ 공사는 등기가 필요한 사항에 관하여는 등기하기 전에는 제3자에게 대항하지 못한다.\n\n[전문개정 2009. 3. 25.]\n\n\n\n제6조 삭제 <2009. 3. 25.>\n\n\n\n제7조(대리ㆍ대행) 정관으로 정하는 바에 따라 사장이 지정한 공사의 직

In [2]:
len(data_list)

5

In [11]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')

In [12]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=data_list, embedding=embedding, collection_name='chroma-korail', persist_directory='./chroma')
database = Chroma(collection_name='chroma-korail', persist_directory='./chroma', embedding_function=embedding)

In [3]:
query = '철도공사는 무슨 목적으로 설치되었어요?'
# retrieved_docs = database.similarity_search(query, k=3)

In [ ]:
# retrieved_docs

[Document(id='a87d08d5-2baa-40c7-880e-033d52e90c90', metadata={'source': './korail.docx'}, page_content='1. 연도별 사업계획 및 예산에 관한 사항\n\n2. 철도서비스 품질 개선에 관한 사항\n\n3. 철도사업계획의 이행에 관한 사항\n\n4. 철도시설ㆍ철도차량ㆍ열차운행 등 철도의 안전을 확보하기 위한 사항\n\n5. 그 밖에 다른 법령에서 정하는 사항\n\n[전문개정 2009. 3. 25.]\n\n\n\n제17조(자료제공의 요청) ①공사는 업무상 필요하다고 인정하면 관계 행정기관이나 철도사업과 관련되는 기관ㆍ단체 등에 자료의 제공을 요청할 수 있다. <개정 2014. 5. 21.>\n\n② 제1항에 따라 자료의 제공을 요청받은 자는 특별한 사유가 없으면 그 요청에 따라야 한다.<신설 2014. 5. 21.>\n\n[전문개정 2009. 3. 25.]\n\n\n\n제18조(등기 촉탁의 대위) 공사가 제9조제1항제4호에 따라 국가 또는 지방자치단체로부터 위탁받은 사업과 관련하여 국가 또는 지방자치단체가 취득한 부동산에 관한 권리를 「부동산등기법」 제98조에 따라 등기하여야 하는 경우 공사는 국가 또는 지방자치단체를 대위(代位)하여 등기를 촉탁할 수 있다. <개정 2011. 4. 12.>\n\n[전문개정 2009. 3. 25.]\n\n\n\n제19조(벌칙) 제8조를 위반한 자는 2년 이하의 징역 또는 2천만원 이하의 벌금에 처한다. <개정 2014. 5. 21.>\n\n[전문개정 2009. 3. 25.]\n\n\n\n제20조(과태료) ① 제8조의2를 위반한 자에게는 500만원 이하의 과태료를 부과한다. <개정 2014. 5. 21.>\n\n② 제1항에 따른 과태료는 국토교통부장관이 부과ㆍ징수한다.<개정 2013. 3. 23.>\n\n[전문개정 2009. 3. 25.]\n\n\n\n\n\n\n\n부칙 <제21065호,2025. 10. 1.>(정부조직법)\n\n제1조(시행일) 이 법은 

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [ ]:
# prompt = f"""
# [Identity]
# - 당신은 한국철도공사 법무팀에서 일하는 전문가입니다.
# - [Context]를 참고해서 사용자의 질문에 답변해주세요.

# [Context]
# {retrieved_docs}

# [Question]
# {query}
# """

In [ ]:
# msg = llm.invoke(prompt)

In [ ]:
# msg.content

'한국철도공사는 철도 운영의 전문성과 효율성을 높임으로써 철도산업과 국민경제의 발전에 이바지하기 위해 설치되었습니다.'

In [8]:
from langchain_classic import hub

prompt = hub.pull("rlm/rag-prompt")

In [15]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = database.as_retriever(search_kwargs={"k": 3})

qa_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = qa_chain.invoke(query)
result

'철도공사는 철도 운영의 전문성과 효율성을 높임으로써 철도산업과 국민경제의 발전에 이바지하기 위해 설치되었습니다.'